# Install the packages

In [ ]:
!pip install langchain
!pip install langchain-google-genai
!pip install google-generativeai
!pip install google-search-results


In [ ]:
!pip install langchain-community
!pip install langchain-google-genai
!pip install streamlit
!pip -q install pyngrok

# Get Gemini API key

In [17]:
from google.colab import userdata
gemini_key=userdata.get('gemini_key')

import os
os.environ["GOOGLE_API_KEY"] = gemini_key



In [18]:
from google.colab import userdata
serpi_key=userdata.get('serpi_key')
# Set up SerpAPI key (Replace with your actual API key)
os.environ["SERPAPI_API_KEY"] = serpi_key

# GET SERP API Key


https://serpapi.com/

- perform email verification

- perform mobile verification

- subscribe free plan


https://serpapi.com/dashboard

In [ ]:

from google.colab import userdata
serpi_key=userdata.get('serpi_key')
# Set up SerpAPI key (Replace with your actual API key)
os.environ["SERPAPI_API_KEY"] = serpi_key


In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import SerpAPIWrapper

# Initialize Gemini model
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

# Set up SerpAPI key (Replace with your actual API key)
os.environ["SERPAPI_API_KEY"] = serpi_key

# Initialize SerpAPI search tool
search = SerpAPIWrapper()
search_tool = Tool(
    name="Google Search",
    func=search.run,
    description="Use this tool to search the internet for real-time information."
)



# Define the agent
agent = initialize_agent(
    tools=[search_tool],
    llm=gemini_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Ask the agent for ICICI Bank share price
query = "Latest ICICI Bank share price in INR on NSE/BSE (National Stock Exchange of India or Bombay Stock Exchange). Provide only the latest number."
response = agent.run(query)
print(response)




> Entering new AgentExecutor chain...
Action: Google Search
Action Input: Latest ICICI Bank share price in INR on NSE/BSE
Observation: {'type': 'finance_results', 'title': 'ICICI Bank Ltd', 'exchange': 'NSE', 'stock': ' ICICIBANK', 'currency': 'INR', 'price': 1259.0, 'previous_close': 1248.9}
Thought:Final Answer: 1259.0

> Finished chain.
1259.0


✅ Agents dynamically search the web—unlike a standard LLM , which relies on pre-existing knowledge.

✅ The right search query matters—small changes in phrasing can make a big difference in accuracy.

✅ SerpAPI is a great alternative when Google Custom Search API has restrictions.

# Create an application


Here’s an improved version of your agent that can:

✅ Fetch the latest stock price of multiple stocks

✅ Summarize recent market news for a company

✅ Compare multiple stock prices side by side

In [ ]:
%%writefile app.py
import os
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper


# Initialize Gemini AI model
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

# Initialize SerpAPI search tool
search = SerpAPIWrapper()
search_tool = Tool(
    name="Google Search",
    func=search.run,
    description="Use this tool to search the internet for real-time stock prices, financial news, and comparisons."
)

# Define the agent with additional capabilities
agent = initialize_agent(
    tools=[search_tool],
    llm=gemini_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Streamlit UI
st.title("📈 AI Stock Price & Market News Bot")
st.write("Fetch the latest stock prices, compare multiple stocks, or get financial news summaries!")

# User selects a query type
query_type = st.selectbox(
    "What do you want to do?",
    ["Fetch Latest Stock Price", "Compare Multiple Stocks", "Get Stock Market News"]
)

if query_type == "Fetch Latest Stock Price":
    stock_name = st.text_input("Enter Stock Name (e.g., ICICI Bank, HDFC Bank)")
    if stock_name and st.button("Get Stock Price"):
        with st.spinner("Fetching latest stock price..."):
            query = f"Latest {stock_name} share price in INR on NSE/BSE (National Stock Exchange of India or Bombay Stock Exchange). Provide only the latest number."
            response = agent.run(query)
        st.success(f"📊 Latest Price: {response}")

elif query_type == "Compare Multiple Stocks":
    stock_names = st.text_input("Enter Stock Names (comma-separated, e.g., ICICI Bank, HDFC Bank, SBI)")
    if stock_names and st.button("Compare Stocks"):
        with st.spinner("Fetching stock comparisons..."):
            query = f"Compare the latest share prices of {stock_names} on NSE/BSE in INR."
            response = agent.run(query)
        st.success(f"📊 Stock Comparison:\n{response}")

elif query_type == "Get Stock Market News":
    company_name = st.text_input("Enter Company Name (e.g., ICICI Bank, TCS)")
    if company_name and st.button("Get Market News"):
        with st.spinner("Fetching latest market news..."):
            query = f"Summarize the latest news and stock performance of {company_name} in the last month."
            response = agent.run(query)
        st.success(f"📰 Latest News:\n{response}")


Writing app.py


In [ ]:
from google.colab import userdata
ngrok_auth=userdata.get('ngrok_auth')


In [ ]:
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token(ngrok_auth)

# create the tunnel
ngrok_tunnel=ngrok.connect(addr="5000",proto="http")
print("Tracking uri:",ngrok_tunnel.public_url)

!streamlit run --server.port 5000 app.py

Tracking uri: https://d720-34-168-182-202.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:5000
  Network URL: http://172.28.0.12:5000
  External URL: http://34.168.182.202:5000

/content/app.py:6: LangChainDeprecationWarning: Importing SerpAPIWrapper from langchain.utilities is deprecated. Please replace deprecated imports:

>> from langchain.utilities import SerpAPIWrapper

with new imports of:

>> from langchain_community.utilities import SerpAPIWrapper
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.utilities import SerpAPIWrapper
/content/app.py:21: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling